In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_ollama import OllamaLLM
from langchain import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain.schema import Document
from langgraph.prebuilt import create_react_agent
import datetime
import numpy as np
import time
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.document_transformers import LongContextReorder
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain.retrievers.multi_query import MultiQueryRetriever


In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH07-MultiQuery")

In [ ]:
OLLAMA_BASE_URL = "http://localhost:11434" # Ollama 서버 주소
OLLAMA_CHAT_MODEL = "ollama-ko-0710:latest" # Ollama에서 실행중인 모델 이름

In [3]:
def get_split_docs():
    # 1. 텍스트 청크 분할 (chunk_size=1000, chunk_overlap=50)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

    # 2. Text loader
    loader = TextLoader("./data/text.txt", encoding="utf-8")
    return loader.load_and_split(text_splitter)

## MMR 알고리즘 특징
- 쿼리(query)와 각 문서(document)의 관련성(유사도)을 높이되
- 이미 선택된 문서와의 중복(유사성)을 최소화하여 서로 다른 내용을 가진 문서를 선택합니다.

In [4]:
def get_retriever(split_docs): 
    model_name = "intfloat/multilingual-e5-large-instruct"
    hf_embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"device": "cuda"},  # cuda, cpu
        encode_kwargs={"normalize_embeddings": True},
    )
    db = FAISS.from_documents(documents=split_docs, embedding=hf_embeddings)
    retriever = db.as_retriever(
        search_type="mmr", search_kwargs={"k": 10, "lambda_mult": 0.25, "fetch_k": 20}
    )
    return retriever

In [5]:

docs = get_split_docs()
retriever = get_retriever(docs)

In [7]:
_docs = retriever.invoke("삼성전자 하반기 실적")
print(len(_docs))

10


In [ ]:
print(_docs[0].page_content)

특히 파운드리 고객사를 확보한 것이 단기 실적에는 영향을 미치지 않지만, 중장기 경쟁력 회복을 위한 기반이 될 것이라고 평가했다. 류 연구원은 "여러 기대감이 모여 밸류에이션이 정상화되는 국면"이라고 분석했다. 
이어 3분기 영업이익은 9조원으로 1년 전보다는 1.9% 감소하지만 직전 분기 대비 91.9% 늘어날 것으로 낙관했다.
미래에셋증권은 15일 보고서에서 삼성전자의 올해와 내년 영업이익 전망치를 기존대비 각각 2.9%, 5.3%씩 올렸다. 동시에 목표 주가를 기존 8만8000원에서 9만6000원으로 9% 높였다.
미래에셋증권은 주가 상향 이유로 내년 메모리 반도체 공급 부족을 꼽았다. 반도체 업체들은 제한된 D램 생산라인을 갖고 있는데, 업황 불확실성 탓에 설비를 크게 늘리지 못했다. 새로 증설한 시설도 대부분 차세대 반도체인 고대역폭 메모리(HBM) 생산에 집중돼 있어 범용 D램의 생산량이 줄면서 가격이 오르고 있다.


## MutiQuery Retriever

LLM 을 사용하여 3가지 쿼리문 생성하여 연결된 retriever 에서 관련된 데이터를 검색 후 병합하여 최종 데이터 context 출력 메커니즘

In [11]:
# ChatOpenAI 언어 모델을 초기화합니다. temperature는 0으로 설정합니다.
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

multiquery_retriever = MultiQueryRetriever.from_llm(  # MultiQueryRetriever를 언어 모델을 사용하여 초기화합니다.
    # 벡터 데이터베이스의 retriever와 언어 모델을 전달합니다.
    retriever=retriever,
    llm=llm,
)

In [12]:
# 쿼리에 대한 로깅 설정
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [18]:
# 문서 출력 도우미 함수
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [16]:
# 질문을 정의합니다.
question = "2025년 삼성전자 하반기 실적 전망은"
# 문서 검색
relevant_docs = multiquery_retriever.invoke(question)

# 검색된 고유한 문서의 개수를 반환합니다.
print(
    f"===============\n검색된 문서 개수: {len(relevant_docs)}",
    end="\n===============\n",
)

# 검색된 문서의 내용을 출력합니다.
print(relevant_docs[0].page_content)

INFO:langchain.retrievers.multi_query:Generated queries: ['2025년 삼성전자의 하반기 실적 예측은 어떻게 되나요?  ', '2025년 삼성전자의 하반기 재무 성과에 대한 전망은 무엇인가요?  ', '2025년 하반기 삼성전자의 실적에 대한 분석이나 예측이 있나요?']


검색된 문서 개수: 15
Q. 2010년대 노키아 몰락후 핀란드가 크게 흔들렸습니다. TSMC 의존도가 절대적으로 높은 대만도 그런 위험에 빠질 가능성이 있을까요?
Q. 2027년 중국의 대만 침공설이 나돌 정도로 중국은 대만 경제 의 앞날에 최대 리스크입니다. 대만내에서는 중국 침공 가능성을 어떻게 보고 있나요?
Q. 한국은 10년째 3만달러 박스에 갇혀 있습니다. 성장률은 잠재성장률에도 못미치는 1% 대로 떨어졌죠.  가장 큰 이유는 무엇일까요?  
Q. 대만 경제의 4만달러 진입 전망이 우리 경제에 주는 시사점은 무엇인가요?
(자세한 내용은 동영상을 시청하시기 바랍니다.)'삼성 AI 포럼 2025' 2일차
전경훈 삼성전자 CTO 강조
 
전경훈 삼성전자 디바이스경험(DX) 최고기술책임자(CTO) 겸 삼성리서치장(사장)이 16일 열린 '삼성 AI 포럼 2025' 2일차 행사에서 발언하고 있다. 삼성전자 제공


In [19]:
pretty_print_docs(relevant_docs)

Document 1:

Q. 2010년대 노키아 몰락후 핀란드가 크게 흔들렸습니다. TSMC 의존도가 절대적으로 높은 대만도 그런 위험에 빠질 가능성이 있을까요?
Q. 2027년 중국의 대만 침공설이 나돌 정도로 중국은 대만 경제 의 앞날에 최대 리스크입니다. 대만내에서는 중국 침공 가능성을 어떻게 보고 있나요?
Q. 한국은 10년째 3만달러 박스에 갇혀 있습니다. 성장률은 잠재성장률에도 못미치는 1% 대로 떨어졌죠.  가장 큰 이유는 무엇일까요?  
Q. 대만 경제의 4만달러 진입 전망이 우리 경제에 주는 시사점은 무엇인가요?
(자세한 내용은 동영상을 시청하시기 바랍니다.)'삼성 AI 포럼 2025' 2일차
전경훈 삼성전자 CTO 강조
 
전경훈 삼성전자 디바이스경험(DX) 최고기술책임자(CTO) 겸 삼성리서치장(사장)이 16일 열린 '삼성 AI 포럼 2025' 2일차 행사에서 발언하고 있다. 삼성전자 제공
----------------------------------------------------------------------------------------------------
Document 2:

당초 한국은행으로 알려졌던 지난해 법인세 납부액 1위도 실제로는 SK하이닉스였다. SK하이닉스는 2024년 연결기준 영업이익 약 23조5,000억 원을 거뒀고, 국내 법인세로 3조6,307억8,300만 원을 납부했다. 2위는 현대차(3조433억 원), 3위는 한국은행(2조5,782억 원), 4위는 기아(2조825억 원) 순이었다. 2023년 별도 영업적자 11조5263억 원을 낸 삼성전자는 2023~2024년 2년 연속 법인세를 내지 않았다.처음으로 19조원 돌파
        이재용 삼성전자 회장의 주식 재산이 처음으로 19조원을 넘어섰다. 지난 11일 18조원을 돌파한 데 이어 3거래일 만이다.
16일 한국CXO연구소에 따르면 이 회장의 주식 재산은 이날 종가 기준 19조152억원으로 집계됐다. 이 회장이 보유한 삼성전자 보통주와 우선주, 삼성물산, 